## User Experience

What happens if the user inputs a song that doesn't exist?
1. <span class="girk">get the song name from the user as an input</span>
2. <span class="girk">play the input song in music embed playe</span>r.
3. <span class="girk">get the audio features for that song from spotify API. using sp.audio_features(trackid) . pay attention to keep  only the audio features columns.</span>
4. <span class="girk">load the stabdardscaler using pickle and use it to scale the new song.</span>
5. <span class="girk">using mode.predict(new scaled audio record for the new song) to predict the cluster (label) for the new song.</span>
6. return random song from the same cluster   that the new song belongs to from your data frame and suggest it to the user.
7.  play it using embedded music player. (edited) 

<b> do a heatmap,with the most importants audiofeatures for every cluster.

# Importing libraries

In [89]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from csv import DictReader
import random
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config

## Starting Spotify API

In [90]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

## Loading Data Frame

In [91]:
df_songs = pd.read_csv (r"C:\Users\Barbara\OneDrive\Documents\GitHub\IHLabs_BV\Week6\audio_features_1.csv")
df_songs

danceability  energy  key  loudness  mode  speechiness  acousticness  \
0            0.795   0.594    0    -6.200     1       0.0748       0.11200   
1            0.693   0.713   10    -4.489     0       0.3510       0.34300   
2            0.400   0.366    8    -9.405     0       0.0597       0.24200   
3            0.543   0.644    1    -5.952     1       0.3330       0.11300   
4            0.824   0.733    1    -5.474     1       0.0613       0.03620   
...            ...     ...  ...       ...   ...          ...           ...   
5626         0.503   0.791    7    -5.177     1       0.0499       0.00241   
5627         0.511   0.766    4    -3.178     1       0.0740       0.01040   
5628         0.799   0.653    2    -5.453     0       0.0370       0.01900   
5629         0.836   0.743   10    -6.305     0       0.0656       0.09950   
5630         0.723   0.772    1    -6.784     1       0.0792       0.00524   

      instrumentalness  liveness  valence    tempo                      id  \
0             0.000000    0.1620    0.409  135.002  0q75NwOoFiARAVp4EXU4Bs   
1             0.000000    0.1020    0.615  137.335  593W4qZOwwdqF6YnimJjL6   
2             0.000000    0.1260    0.161   73.313  6gU9OKjOE7ghfEd55oRO57   
3             0.000000    0.0733    0.706   97.410  5wJL4o7k8m02m6ZM9KT0ir   
4             0.000000    0.3250    0.395   97.972  7FYH5AW3bVfZHJIQpq3UOA   
...                ...       ...      ...      ...                     ...   
5626          0.000000    0.2980    0.417  173.921  4shULiz3P7luscRsQobtDN   
5627          0.004790    0.3900    0.603  160.866  5f20q1GsH6sr5tD91jgbyP   
5628          0.000000    0.0935    0.236  104.019  3vySEUpD0tc801F2WZDLYw   
5629          0.000000    0.3350    0.722  108.966  1PckUlxKqWQs3RlWXVBLw3   
5630          0.000012    0.3670    0.580  102.004  4cacyP5c3PMlfnyjpg13xW   

      duration_ms  time_signature  cluster  
0          275080               4       10  
1          266080               4        3  
2          272187               3        4  
3          227147               4        7  
4          284387               4       10  
...           ...             ...      ...  
5626       166037               4        6  
5627       159129               4        6  
5628       215281               4        0  
5629       191822               4        3  
5630       146471               4        2  

[5631 rows x 15 columns]

## Getting the song name from user as an Input

In [239]:
#Collecting The Input As A Variable:#
#nombre de la cancion, buscar el nombre en el df y tomr el id del track
song_name = input('Please enter the song name : ') 
song_name = song_name.lower()
results = sp.search(q="track:"+song_name,limit = 3, market = "US") #to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.

Please enter the song name : bad guy


In [262]:
list_of_artists=[]
for index in range(0,len(results["tracks"]["items"])): 
    list_of_artists.append(results['tracks']['items'][index]['artists'][0]['name'])
print(list_of_artists)
choice_artist = input('Which artist do you want to listen: 1, 2 or 3?????  ')
final_artist = results['tracks']['items'][int(choice_artist)-1]['artists'][0]['name']


['Billie Eilish', 'Falling In Reverse', 'Bowling For Soup']
Which artist do you want to listen: 1, 2 or 3?????  1


In [263]:
track_id_json = sp.search(q="track:"+song_name+" artist:"+final_artist,limit=1)
track_id = track_id_json['tracks']['items'][0]['id']

##  Input song in music embed player

In [264]:
from IPython.display import IFrame
#track_id = "1rfORa9iYmocEsnnZGMVC4"

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

# Audio features for the user song from spotify API

In [273]:
df_input = pd.DataFrame(sp.audio_features(track_id))
df_input = df_input.drop(['type', 'uri', 'track_href', 'analysis_url','id'], axis=1)

## load the standardscaler using pickle and use it to scale the new song.

In [274]:
def load_pickle(filename = "file_name.pickle"):
    try:
        with open(filename,"rb") as f:
            return pickle.load(f)
    except FileNotFoundError:
        print("I don't find your file, sorry, try again!")

In [275]:
scaler = load_pickle(filename = r"C:\Users\Barbara\OneDrive\Documents\GitHub\IHLabs_BV\Week6\scaler.pickle" )
model_kmeans = load_pickle(filename = r"C:\Users\Barbara\OneDrive\Documents\GitHub\IHLabs_BV\Week6\kmeans.pickle" )

## using mode.predict(new scaled audio record for the new song) to predict the cluster (label) for the new song.

In [277]:
columns = df_input.columns
audio_features_scaled = scaler.transform(df_input)
audio_features_scaled = pd.DataFrame(audio_features_scaled, columns= columns)
labels = model_kmeans.predict(audio_features_scaled)

## return random song from the same cluster that the new song belongs to from your data frame and suggest it to the user

In [278]:
recommendation=df_songs[df_songs['cluster']==labels[0]].sample()
track_id_recom = recommendation['id'].iloc[0]

In [279]:
from IPython.display import IFrame
#track_id = "1rfORa9iYmocEsnnZGMVC4"
IFrame(src="https://open.spotify.com/embed/track/"+track_id_recom,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )